In [ ]:
import os
PROJECT_DIR = os.path.abspath(os.path.abspath('') + "/../..")
os.chdir(PROJECT_DIR)

from basicts import launch_runner, BaseRunner

In [ ]:

# MODEL_NAME = 'MLP'
# MODEL_NAME = 'ConvLSTM'
# MODEL_NAME = 'TimesNet'
# MODEL_NAME = 'FEDformer'
MODEL_NAME = 'STID'
# DATASET_NAME = 'Pulse'
DATASET_NAME = 'Rainfall_USA'
BATCH_SIZE = 128
GPUS = '1'

cfg_path = 'baselines/{0}/{1}.py'.format(MODEL_NAME, DATASET_NAME) # NOTE: use relative path
# ckpt_path = '' # NOTE: use relative path

# ckpt_path = 'checkpoints/GWNet_100/be346592bdaa1753d20c636c5f206816/GWNet_076.pt' # NOTE: use relative path
ckpt_path = f'../checkpoints/{MODEL_NAME}_100/e9dced0d66dba86f1f2122f410404761/{MODEL_NAME}_best_val_MAE.pt'

In [ ]:

def inference(cfg: dict, runner: BaseRunner, ckpt: str = None, batch_size: int = 1):
    # init logger
    runner.init_logger(logger_name='easytorch-inference', log_file_name='validate_result')
    # init model
    cfg.TEST.DATA.BATCH_SIZE = batch_size
    runner.model.eval()
    runner.setup_graph(cfg=cfg, train=False)
    # load model checkpoint
    runner.load_model(ckpt_path=ckpt)
    # test
    runner.init_test(cfg)
    global results
    results = runner.test()

launch_runner(cfg_path, inference, (ckpt_path, BATCH_SIZE), devices=GPUS)


In [ ]:
prediction = results['prediction'].detach().cpu().numpy()
target = results['target'].detach().cpu().numpy()
inputs = results['inputs'].detach().cpu().numpy()

In [ ]:
print(prediction.shape) # num_samples, len_prediction, num_time_series, num_channels
import random
sample_id = random.randint(0, prediction.shape[0])
time_series_id = random.randint(0, prediction.shape[2]-1)
channel_id = random.randint(0, prediction.shape[3]-1)

In [ ]:
prediction.shape # num_samples, len_prediction, num_time_series, num_channels
import random
sample_id = random.randint(0, prediction.shape[0])
time_series_id = random.randint(0, prediction.shape[2]-1)
channel_id = random.randint(0, prediction.shape[3]-1)

# plot history, targets, and predictions
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(10, 5))

plt.plot(np.arange(inputs.shape[1]), inputs[sample_id, :, time_series_id, channel_id], label='history')
plt.plot(np.arange(inputs.shape[1], inputs.shape[1]+target.shape[1]), target[sample_id, :, time_series_id, channel_id], label='target')
plt.plot(np.arange(inputs.shape[1], inputs.shape[1]+prediction.shape[1]), prediction[sample_id, :, time_series_id, channel_id], label='prediction')
plt.legend()
plt.show()
